# Sprint
## ディープラーニングフレームワーク1

## 1.このSprintについて

### Sprintの目的
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

### どのように学ぶか
TensorFLowのサンプルコードを元に、これまで扱ってきたデータセットを学習していきます。

## 2.コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。


tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

### 【問題1】
#### スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）


- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

- forward, backwardの実装が必要
- 最適化手法の実装が必要

### データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードでは`Iris.csv`が同じ階層にある想定です。


[Iris Species](https://www.kaggle.com/uciml/iris/data)


目的変数は`Species`ですが、3種類ある中から以下の2種類のみを取り出して使用します。

```
Iris-versicolor
Iris-virginica
```

### 【問題2】
#### スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [1]:
#《サンプルコード》
# ＊バージョン1.5から1.14の間で動作を確認済みです。
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [2]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [3]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, loss : 2.9396, val_loss : 10.0471, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 0.0000, val_loss : 4.4855, acc : 1.000, val_acc : 0.750
Epoch 2, loss : 1.4334, val_loss : 7.3675, acc : 0.750, val_acc : 0.562
Epoch 3, loss : 0.0000, val_loss : 7.1097, acc : 1.000, val_acc : 0.750
Epoch 4, loss : 1.4344, val_loss : 10.7628, acc : 0.750, val_acc : 0.500
Epoch 5, loss : 0.0000, val_loss : 6.0986, acc : 1.000, val_acc : 0.750
Epoch 6, loss : 0.0004, val_loss : 6.3735, acc : 1.000, val_acc : 0.688
Epoch 7, loss : 0.0000, val_loss : 2.4152, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.0000, val_loss : 3.2437, acc : 1.000, val_acc : 0.812
Epoch 9, loss : 0.0000, val_loss : 0.1243, acc : 1.000, val_acc : 0.938
test_acc : 0.950


- 重み・バイアスの初期化

サンプルコードでは`tf.Variable`型で管理を行っている。

sessionを開始した際に一度だけ`tf.global_variables_initializer()`を実行することで、Variable（重みやバイアス）を初期化している。

- エポックのループ

スクラッチ実装の際と同じようにforループを使って実装している。

高レベルAPIである`tf.estimator`を使う場合だと`num_epochs`引数で指定できるらしい。

- forward, backward
- 最適化手法

最適化手法は実装済みで、損失関数をminimizeするように指定することでweightsとbiasesが最適化される。

実際にsession内でrunする際には、計算グラフの目的地点である`train_op`や`accracy`などに対して行うだけで、それより前の計算グラフは自動的に実行される。

スクラッチ実装の時のようにforループを使って出力を保存しながらレイヤーでの計算を行う必要はない。


## 3.他のデータセットへの適用

これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。


- Iris（3種類全ての目的変数を使用）
- House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

### 【問題3】
#### 3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


[Iris Species](https://www.kaggle.com/uciml/iris/data)


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


**《ヒント》**


以下の2箇所は2クラス分類特有の処理です。

```python
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
```

```python
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
```

メソッドは以下のように公式ドキュメントを確認してください。


[tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)


[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/sign)


＊tf.signとtf.math.signは同じ働きをします。

In [4]:
df = pd.read_csv(dataset_path)
display(df)
X = np.array(df.iloc[:, 1:5])
y = np.array(pd.get_dummies(df.iloc[:, 5]))
display(X.shape)
display(y.shape)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


(150, 4)

(150, 3)

In [5]:
y = y.astype(np.int)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [6]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1], 0, 0.1)),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2], 0, 0.1)),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes], 0, 0.1))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1], 0, 0.1)),
        'b2': tf.Variable(tf.random_normal([n_hidden2], 0, 0.1)),
        'b3': tf.Variable(tf.random_normal([n_classes], 0, 0.1))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    layer_output = tf.nn.softmax(layer_output)
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = -tf.reduce_sum(Y*tf.log(logits+1e-12))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_val_loss = 0
        total_val_acc = 0
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            total_val_loss += val_loss
            total_val_acc += val_acc
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        total_val_loss /= total_batch
        total_val_acc /= total_batch
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 4.9765, val_loss : 15.4531, acc : 0.833, val_acc : 0.792
Epoch 1, loss : 3.9537, val_loss : 10.6266, acc : 0.500, val_acc : 0.958
Epoch 2, loss : 2.1317, val_loss : 9.4606, acc : 1.000, val_acc : 0.750
Epoch 3, loss : 0.9530, val_loss : 7.3960, acc : 1.000, val_acc : 0.833
Epoch 4, loss : 0.5371, val_loss : 4.9845, acc : 1.000, val_acc : 0.917
Epoch 5, loss : 0.3341, val_loss : 4.2910, acc : 1.000, val_acc : 0.958
Epoch 6, loss : 0.0850, val_loss : 6.0582, acc : 1.000, val_acc : 0.833
Epoch 7, loss : 0.0529, val_loss : 4.2008, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.0367, val_loss : 4.0703, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.0219, val_loss : 4.2483, acc : 1.000, val_acc : 0.875
test_acc : 0.967


### 【問題4】
#### House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)


この中の`train.csv`をダウンロードし、目的変数として`SalePrice`、説明変数として、`GrLivArea`と`YearBuilt`を使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。



In [7]:
df = pd.read_csv('train.csv')
display(df)
X = np.array(df[['GrLivArea', 'YearBuilt']])
y = np.array(df['SalePrice']).reshape(-1, 1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
y_train = np.log(y_train)
y_val = np.log(y_val)
y_test = np.log(y_test)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(934, 2) (934, 1)
(234, 2) (234, 1)
(292, 2) (292, 1)


In [10]:
# ハイパーパラメータの設定
learning_rate = 0.03
batch_size = 10
num_epochs = 10
n_hidden1 = 5
n_hidden2 = 5
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, 1])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, 1]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([1]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# 決定係数
def r2_score(y_true, y_pred):
    residual_fluctuation = tf.reduce_sum(tf.square(y_true-y_pred))
    total_fluctuation = tf.reduce_sum(tf.square(y_true-tf.reduce_mean(y_true)))
    r2 = 1 - (residual_fluctuation/total_fluctuation)
    return r2
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.losses.mean_squared_error(Y, logits)
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 決定係数
r2 = r2_score(Y, logits)
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_val_loss = 0
        total_loss = 0
        total_val_r2_score = 0
        total_r2_score = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, r2_score = sess.run([train_op, loss_op, r2], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_r2_score = sess.run([loss_op, r2], feed_dict={X: X_val, Y: y_val})
            total_val_loss += val_loss
            total_loss += loss
            total_val_r2_score += r2_score
            total_r2_score += val_r2_score
        total_loss /= total_batch
        total_val_loss /= total_batch
        total_r2_score /= total_batch
        total_val_r2_score /= total_batch
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, r2 : {:.4f}, val_r2 : {:.4f}".format(epoch, total_loss, total_val_loss, total_r2_score, total_val_r2_score))
    test_loss, test_r2_score = sess.run([loss_op, r2], feed_dict={X: X_test, Y: y_test})
    print("test_loss : {:.4f}, tesr_r2 : {:.4f}".format(test_loss, test_r2_score))

Epoch 0, loss : 43.1601, val_loss : 40.8252, r2 : -260.8171, val_r2 : -399.3053
Epoch 1, loss : 0.7864, val_loss : 0.6923, r2 : -3.4399, val_r2 : -5.9386
Epoch 2, loss : 0.2326, val_loss : 0.2129, r2 : -0.3656, val_r2 : -1.0160
Epoch 3, loss : 0.0945, val_loss : 0.0875, r2 : 0.4389, val_r2 : 0.2105
Epoch 4, loss : 0.0668, val_loss : 0.0621, r2 : 0.6019, val_r2 : 0.4453
Epoch 5, loss : 0.0583, val_loss : 0.0540, r2 : 0.6536, val_r2 : 0.5148
Epoch 6, loss : 0.0550, val_loss : 0.0508, r2 : 0.6739, val_r2 : 0.5451
Epoch 7, loss : 0.0538, val_loss : 0.0498, r2 : 0.6805, val_r2 : 0.5567
Epoch 8, loss : 0.0536, val_loss : 0.0497, r2 : 0.6816, val_r2 : 0.5539
Epoch 9, loss : 0.0533, val_loss : 0.0500, r2 : 0.6791, val_r2 : 0.5468
test_loss : 0.0577, tesr_r2 : 0.6191


回帰問題の出力層においては**活性化関数を用いない**、もしくは**恒等関数**（出力と入力が同じ）を利用することになる。

また、モデルの評価指標として正解率ではなく誤差や決定係数が用いられる。今回は**MSE**と決定係数を利用した。

### 【問題5】
#### MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [11]:
# データのロード
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print(X_train.shape) # (48000, 784)
print(X_val.shape)
print(X_test.shape)

Using TensorFlow backend.
(48000, 784)
(12000, 784)
(10000, 784)


In [12]:
y_train = np.identity(10)[y_train]
y_val = np.identity(10)[y_val]
y_test = np.identity(10)[y_test]
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(48000, 10)
(12000, 10)
(10000, 10)


In [13]:
# ハイパーパラメータの設定
import time
learning_rate = 0.005
batch_size = 200
num_epochs = 10
n_hidden1 = 300
n_hidden2 = 150
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    initializer = tf.initializers.he_normal()
    weights = {
        'w1': tf.get_variable(name='w1', shape=[n_input, n_hidden1], initializer=initializer),
        'w2': tf.get_variable(name='w2', shape=[n_hidden1, n_hidden2], initializer=initializer),
        'w3': tf.get_variable(name='w3', shape=[n_hidden2, n_classes], initializer=initializer)
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    layer_output = tf.nn.softmax(layer_output)
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        start = time.time()
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_val_loss = 0
        total_val_acc = 0
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            total_val_loss += val_loss
            total_val_acc += val_acc
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        total_val_loss /= total_batch
        total_val_acc /= total_batch
        elapse = time.time() - start
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}, {:.3f}s".format(epoch, loss, val_loss, acc, val_acc, elapse))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 343.1610, val_loss : 20420.8223, acc : 0.750, val_acc : 0.759, 19.271s
Epoch 1, loss : 305.0857, val_loss : 18292.0723, acc : 0.930, val_acc : 0.936, 20.198s
Epoch 2, loss : 303.3423, val_loss : 18152.8398, acc : 0.945, val_acc : 0.949, 20.600s
Epoch 3, loss : 301.1102, val_loss : 18067.0195, acc : 0.955, val_acc : 0.956, 22.269s
Epoch 4, loss : 297.9541, val_loss : 18103.1445, acc : 0.970, val_acc : 0.952, 20.922s
Epoch 5, loss : 297.3806, val_loss : 18074.5527, acc : 0.975, val_acc : 0.955, 19.845s
Epoch 6, loss : 298.0678, val_loss : 18055.5469, acc : 0.970, val_acc : 0.956, 21.898s
Epoch 7, loss : 301.6006, val_loss : 18044.9707, acc : 0.950, val_acc : 0.957, 20.104s
Epoch 8, loss : 301.1674, val_loss : 17983.2617, acc : 0.955, val_acc : 0.962, 22.378s
Epoch 9, loss : 299.3402, val_loss : 18091.4062, acc : 0.965, val_acc : 0.953, 21.722s
test_acc : 0.960


正解率96%と十分な精度を出すことができた。